## This notebook contains a **tutorial** followed by **Problem 3 of Homework 3**, both of which are based on PyTorch.

### To complete this problem, please upload this .ipynb (Notebook) file **to Google Colab.** (You'll need to use Colab because it supports GPU and has all the necessary packages pre-installed.) When you're finished, go to "File -> Download .ipynb" and submit your .ipynb notebook as a supplemental file.

### This notebook has instructions for several homework problems.  ***You must write your answers to these problems in your PDF submission file.*** Your Colab notebook will not be graded, so please make sure you transfer all deliverables (including plots and written work) to your PDF.  

In [ ]:
import torch
import numpy as np
import math

So far in CS 181, we've been using numpy and scikit-learn to write ML code. In this tutorial, we'll introduce you to **PyTorch**, the ML library of choice for industry and academia.

**Before working through this notebook, make sure that the runtime type is set to GPU.** Follow Edit -> Notebook settings -> and select GPU as the hardware accelerator.

# Tutorial: Getting to Know PyTorch - Key Concepts

### Why PyTorch?

PyTorch...
*   Provides flexible building blocks for building customizable models, including neural network layers, activation functions, and optimization algorithms
*   Can automatically compute gradients, making backpropagation a breeze!
*   Can accelerate computation on GPUs

## Tensors

Tensors are the basic units of storage and computation in PyTorch. They are very similar to NumPy arrays, and follow the same syntax for indexing and for most arithmetic operations.


In [ ]:
# Basic tensor operations
x_tensor = torch.tensor([1, 2, 3, 4, 5])
y_tensor = torch.ones((2, 5))
prod = x_tensor * y_tensor[1,:]
print(prod, prod.shape) # Use .shape to view shape just as in NumPy

In [ ]:
# Easy conversion between PyTorch and Numpy
x_numpy = np.array([1, 2, 3, 4, 5])
assert (x_tensor.numpy() == x_numpy).all()
assert (x_tensor == torch.from_numpy(x_numpy)).all()

What makes tensors **different** from NumPy arrays is:

1. They support GPU acceleration.
2. They contain additional information (accessed as attributes, which we will see later) which facilitate the easy building and training of neural networks, via **computation graphs** and **gradients**.

## A motivating example

To see the advantages of PyTorch in action, let's first take a look at how one might train a simple model using only NumPy:

In [ ]:
import numpy as np
import math

# Create 2000 random input and output data points
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Step 0 - Initialization: Initialize model parameters (randomly, in this example)
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Step 1 - Forward Pass: compute predicted y according to the following model:
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Step 2 - Compute Loss (MSE in this example)
    loss = np.square(y_pred - y).sum() # Sum across 2000 training points
    if t % 100 == 0:
        print(f'Epoch {t}: loss {loss}')

    # Step 3 - Differentiate: Use backprop (chain rule) to compute gradients
    # of the loss function with respect to a, b, c, d
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum() # Sum across 2000 training points
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Step 4 - Optimize: Update weights (vanilla GD in this example)
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

# From: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

Each step involved in training a model is coded very explicitly in this example:
0. Initialization: Initialize model parameters (i.e. weights)
1. Forward Pass: Compute predictions
2. Compute Loss
3. Differentiate: Compute gradients via backpropagation (chain rule)
4. Optimize: Update weights using gradients

Notice how:
- Every model parameter is stored as a variable, which contains a weight value to be continuously updated through training
- The calculation of the gradient of the loss function with respect to the model parameters is coded explicitly and thus *model dependent*
<!-- - The gradient of the loss function needs to be computed with respect to the model parameters explicitly using the chain rule -->
- Each model parameter's gradient is used to update the weight value according to a particular update rule which is also explicitly implemented - in this example, standard gradient descent (GD).

Steps 0 and 1 might not be so difficult using a basic package such as NumPy; however, one could imagine that steps 2 and 3 would quickly become too burdensome to implement manually for models with more parameters, both because of 1) the large number of parameters and 2) the fact that the explicit implementation of the chain rule and update rule will look different every time depending on the the model structure and optimization algorithm.

This is where PyTorch's **automatic differentiation** engine, **Autograd**, is useful. As the name implies, PyTorch can take care of computing gradients and applying optimization update rules automatically under the hood, such that the code we write to invoke these two operaations will look (very nearly) the same across all models.

## Computation Graphs and Autograd

<!-- Old paragraphs, commented out to preserve in case we want to bring back language -->
<!-- Autograd, PyTorch's automatic differentiation engine, is one of the best things that PyTorch has to offer when it comes to training neural networks! Recall that in order to compute gradients for weights in a neural network, you have to backpropagate errors at each node. This is computationally intensive even for a one or two layers and would be a nightmare to implement manually for very deep networks. -->
<!-- PyTorch tensors can keep track of a *computation graph* whenever operations are performed on/between tensors. Autograd performs backprop on this computation graph, so all the steps to compute the gradient are abstracted away. To perform a gradient step update, you simply have to call built-in PyTorch functions that computes the derivative of the loss and (if you're using gradient descent) takes a step toward a gradient of zero. -->

To implement automatic differentiation, PyTorch keeps track of a **computation graph** which logs every operation used to construct a certain output or mathematical expression (more on computation graphs in the [Appendix](#Appendix)). Computation graphs are created automatically whenever mathematical operations are performed using tensors as inputs. Using this graph, **Autograd** can perform backpropagation by repeated applications of the chain rule, and all of these differentiation steps are abstracted away from the user. To perform weight updates, we simply call built-in PyTorch optimizer functions which implement different update rules using the gradients computed during backpropagation.

As we work through the following example, keep in mind that computation graphs in PyTorch are *implicitly* constructed when one builds mathematical expressions. That is, one will never have to write code expressly for the purpose of building such a graph; rather, the computation graph will be assembled automatically using whichever Tensors or operations are appropriately "earmarked" for tracking (see `requires_grad` below).

Let's start with a simple example of autograd. (Original example from https://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html.)


In [ ]:
# Create tensors
x = torch.randn(2, 2)
y = torch.randn(2, 2)
print('x:', x)
print('y:', y)

Tensors have a ```requires_grad``` field. When ```requires_grad == True``` for a certain tensor, any subsequent operations that this tensor is an input to will be incorporated into a computation graph. Autograd will then be able to automatically compute partial derivatives *with respect to this tensor*. (Derivative of what function? More on that [later](#step5).) 

For any tensors which are the outputs of operations tracked in the computation graph, the ```grad_fn``` attribute will indicate the operation that created the tensor, represented as a `Function` object. (It is these `Function`s which encode the computation graph DAG [Directed Acyclic Graph].)

**Important: If some starting tensors have requires_grad = True, then every tensor resulting from operations of those starting tensors will also have requires_grad = True.**



In [ ]:
# By default, user created Tensors have ``requires_grad=False``
print('x and y requires_grad:', x.requires_grad, y.requires_grad)
z = x + y
# So you can't backprop through z
print('z grad_fn:', z.grad_fn)

In [ ]:
# ``.requires_grad_( ... )`` changes an existing Tensor's ``requires_grad`` flag in-place.
# The input flag defaults to ``True`` if not given.
x = x.requires_grad_()
y = y.requires_grad_()
# z contains enough information to compute gradients, as we saw above
z = x + y
print('z grad_fn:', z.grad_fn)
# If any input to an operation has ``requires_grad=True``, so will the output
print('z requires_grad:', z.requires_grad)

We can also detach a tensor from its computation graph:

In [ ]:
# Now z has the computation history that relates itself to x and y
# Can we just take its values, and **detach** it from its history?
new_z = z.detach()

# ... does new_z have information to backprop to x and y?
# NO!
print('new_z grad_fn:', new_z.grad_fn)
# And how could it? ``z.detach()`` returns a tensor that shares the same storage as ``z``, but with the computation history forgotten.
# It doesn't know anything about how it was computed.
# In essence, we have broken the Tensor away from its past history

Now that we've been introduced to `requires_grad`, `grad_fn` and the computation graph....

#### Question: Where should I actually set requires_grad to make everything work?
#### Short Answer: In any tensors which represent model parameters or weights.

Long Answer:

- If you are initializing a parameter(s)/weight(s) by creating an individual tensor(s) manually, you have to set its `requires_grad = True` argument. 
- If you are initializing a parameter(s)/weight(s) by using built-in PyTorch methods from packages such as nn.Layer and nn.Parameter, the internally initialized tensors should already have `requires_grad` set to `True`.


## GPU Acceleration

GPUs are built for parallel processing — they can handle thousands of operations at the same time — which makes them great for computationally-intensive neural nets.

In [ ]:
# To run tensor operations on GPU in Colab, go to Edit -> Notebook settings -> and select GPU as the hardware accelerator.
# If GPU is enabled, the following assertion should pass.
assert torch.cuda.is_available()

In [ ]:
# When creating a tensor, set its device to use CUDA (a GPU parallel computing platform), as shown below.
# What this means is that we are telling PyTorch to physically initialize the relevant 
# variable in GPU memory.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
x = torch.rand(5, 5, device=device)
y = torch.rand(5, 5, device=device)
x * y

## Defining a Model

* The ```nn``` package defines basic neural network layers (or Modules, in PyTorch-speak) as well as common loss functions. Modules take a tensor as input, apply some computation/function to it, and then output the new tensor. 

* The ```optim``` package defines common optimization algorithms such as Adam, RMSProp, and AdaGrad. These algorithms are all variations of stochastic (randomized) gradient descent, and they may differ in terms of how the gradient steps are taken and how the learning rate adapts over epochs. (More details here: https://pytorch.org/docs/stable/optim.html.)

### Example: Cubic Model
Let's say we want to predict y's modeled as ```ax + bx^2 + cx^3```, and our goal is to learn the weights ```a```, ```b```, and ```c```. (Original example from https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#id21.) 

### Step 1: Generate Data

In [ ]:
# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p) 
print(xx.shape)
# Unsqueeze inserts a dimension of size one at the specified location; the tensor
# is reshaped to (2000, 1). Since p is a tensor, pow is applied according to
# broadcasting rules; xx has a final shape of (2000, 3). See documentation for 
# more on broadcasting; PyTorch supports NumPy broadcasting semantics.

<a name="step2"></a>

### Step 2: Define Model (using nn package)

In [ ]:
# Use the nn package to define our model as a sequence of layers.
# nn.Sequential is a Module which contains other Modules, and applies them in sequence to produce its output.
# nn.Linear computes the 3 input features (x, x^2, and x^3) into 1 output feature using a linear function,
# holding internal tensors for its weight and bias.
# nn.Flatten flattens the output of the linear layer to a 1D tensor to match the shape of y.
# In this case, the first dim it flattens is at index 0, and the last dim it flattens is at index 1.

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

### Step 3: Define Loss Function (using nn package)

In [ ]:
# In this case, we will use Mean Squared Error. 
# You can either implement loss functions manually, or
# you can use PyTorch's implementations of commonly-used loss functions.
loss_fn = torch.nn.MSELoss(reduction='sum')

### Step 4: Define Optimizer (using optim package)

Use the optim package to define an Optimizer that will update the weights of the model for us. Here we will use RMSprop; the optim package contains many other optimization algorithms. The first argument to the RMSprop constructor tells the optimizer which Tensors it should update.

In general, what distinguishes different optimizers is the way that they use gradients to define an update rule or scheme for their model weights. It follows then that any optimizer will always need the weights to be updated, and their corresponding gradients. In PyTorch, both of these quantities are stored together under model parameter variables. In particular, if `var` is a model weight, `var.grad` will contain a gradient with respect to `var` after backpropagation is performed. More in the next code cell.

In [ ]:
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

<a name="step5"></a>

### Step 5: Train the Model

Every time you train a model, be sure to include these 3 short but important lines that do much of the heavy lifting for you!

1.   `optimizer.zero_grad()`: Zeroes out gradients from previous iterations. Otherwise, over the epochs, gradients will accumulate (sum) in `parameter.grad` of each `parameter` which was passed to the optimizer as an argument.
2.   `loss.backward()`: Does all the backpropagation in one fell swoop (takes the derivative of the loss). We call `.backward()` on the loss tensor, which has the effect of taking the partial derivative of the loss with respect to the model parameters. For a model parameter (weight) `w`, dloss/dw will be stored in `w.grad` after `loss.backward()`.
3. `optimizer.step()`: Based on the previously specified optimizer algorithm and learning rate, updates the parameters in the direction of the gradient. This represents one iteration of the weight update rule.




In [ ]:
num_epochs = 2000
for epoch in range(num_epochs):

    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if epoch % 100 == 99:
        print(epoch, loss.item())

    # Before the backward pass, zero all the gradients for the variables it will update
    # (these are the learnable weights of the model). This is because by defaults, 
    # gradients from previous iterations won't be overwritten. 
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # After taking the backward pass, you can see the computed gradient for each parameter
    if epoch % 100 == 99:
      # The model implicitly stores the parameters, as shown below:
      for i, param in enumerate(model.parameters()):
        # param.grad refers to d_loss/d_parameter
        print('parameter', i, 'gradient', param.grad)

    # Calling the step function on an Optimizer makes an update to its parameters
    optimizer.step()

linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

# **Problem 3, Part 1: PyTorch Concept Questions**

*Write up your answers to these problems in your PDF submission.  Any work included in your Colab notebook will not be graded.*

These questions are intended to be a straightforward check on some fundamental PyTorch concepts. You are encouraged to use the PyTorch documentation and any other online resources to answer these questions.  Please write no more than 3 sentences to answer each of the below questions.

# Problem 3.1
Where are gradients stored in PyTorch and how can they be accessed?
# Problem 3.2
In the model defined in [Step 2](#step2) in the tutorial section, how many weights does the model have, and which layer do they come from?
# Problem 3.3
Conceptually, what is a fully-connected layer in a neural network, and how is it represented in PyTorch?


# **Problem 3, Part 2: Training a Neural Network on CIFAR-10**

In this part, you will train two neural networks to perform multi-class classifications on the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). This dataset consists of 32x32 images belonging to 10 classes. In particular:



# Problem 3.4
**Initialize:** Complete the Part2NeuralNetwork class with 3 fully connected (linear) layers using ReLU activation functions, and with the hidden layers having 1000 nodes each.

Here, we ask you to define a model class which inherits from `nn.Module` rather than using `nn.Sequential` as we saw before in [Step 2](#step2). This is done by initializing weights in `__init__()` and manually implementing the forward pass in `forward()`. Feel free to reference documentation.

Please copy and paste your neural network class into your PDF writeup.  We recommend using a LaTeX library such as [listings](https://ctan.org/pkg/listings?lang=en) to ensure your code renders correctly.

*Include in your submission: Your neural network class code.*

# Problem 3.5

**Train:** Complete the training code for the model. Use [cross-entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) and an optimizer of your choice.

Train your model for 10 epochs. Every epoch, log the train and test set loss.  Create a plot that displays both the model's train and the test set loss across epochs.

*Include in your submission: 1 plot which has both the train and test loss vs. number of epochs.*

# Problem 3.6

**Evaluate:** To evaluate your model, compute the following metrics on the train and test set, including them in your write-up.  For each of the following metrics, use the model's "hard assignment" of labels, i.e. your model's predictions should assign each data point to only 1 class:

* The model's classification accuracy
* The model's [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) for each of the 10 classes

What kinds of errors do you believe that the trained model is making?  Use evidence (such as the above metrics, example misclassified images, or any other metrics of your choice) to support your claims.

*Include in your submission: train/test accuracy, train/test precision/recall for 10 classes, and evidence to support your hypothesis about the model's errors.*

# Problem 3.7

**Explore:** Create a new neural network with at least 1 architectural modification.   Some things you can explore are adding or removing layers, changing the number of nodes at each layer, or experimenting with convolutional and pooling layers (see Appendix). The only requirement is that your model attain at least 50% test set accuracy after training for 10 epochs.  This part of the problem is intentionally open-ended, and we encourage you to explore!

For your new neural network, include a plot of train and test set loss in your writeup.  Calculate your model's train/test accuracy and precision/recall for the 10 classes.

In your writeup, copy and paste your modified neural network class and describe the architectural changes you made.  Write at least 1 sentence about why you hypothesize your new neural network performed better or performed worse than the network in Part 3.4.

*Include in your submission: Your neural network class code, 1 plot of train/test loss, metrics from Part 3.6 for this new neural net, and explanation of performance.*


In [ ]:
## Download CIFAR-10 dataset 

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./part_2_data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./part_2_data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# TODO - Complete Part2NeuralNetwork. Include class into your PDF submission!

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class Part2NeuralNetwork(nn.Module):
    def __init__(self):
        super(Part2NeuralNetwork, self).__init__()
        ## TODO: Define your neural network layers here!
        ## Importantly, any modules which initialize weights should be initialized
        ## as member variables here. 
        ## Note: Keep track of the shape of your input tensors in the training
        ## and test sets, because it affects how you define your layers!
        ## You might find this resource helpful:
        ## https://towardsdatascience.com/pytorch-layer-dimensions-what-sizes-should-they-be-and-why-4265a41e01fd

    def forward(self, x):
        ## TODO: This is where you should apply the layers defined in the __init__
        ## method and the ReLU activation functions to the input x.
        return x

# Display model architecture
model = Part2NeuralNetwork()
model.to(device)

In [ ]:
# TODO - Complete NN training

# Reinitialize to ensure we are training a new model
model = Part2NeuralNetwork()
model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

train_losses = []
test_losses = []

for epoch in range(10):
    running_loss = 0.0
    model.train(True)
    for i, data in enumerate(trainloader, 0):
        # Get inputs; data is a list of [inputs, labels]
        xs, ys = data
        xs = xs.to(device)
        ys = ys.to(device)

        ## TODO: optimize model parameters

        running_loss += loss.item()
    train_losses.append(running_loss / 50000)
    print('[Epoch %d] average train loss: %.3f' % (epoch + 1, running_loss / 50000))
    
    model.train(False)
    running_test_loss = 0.0
    ## TODO: calculate test loss in similar fashion

    test_losses.append(running_test_loss / 10000)
    print('[Epoch %d] average test loss: %.3f' % (epoch + 1, running_test_loss / 10000))

print('Finished Training')

In [ ]:
# Use this function to compute your model accuracy!
# Note that the dataloader argument can take trainloader or testloader
def compute_accuracy(model, dataloader):
    correct = 0
    total = 0
    model.train(False)
    with torch.no_grad():
        for data in dataloader:
            xs, ys = data
            xs = xs.to(device)
            ys = ys.to(device)
            y_preds = model(xs)
            _, predicted = torch.max(y_preds.data, 1)
            total += ys.size(0)
            correct += (predicted == ys).sum().item()
    print('Model accuracy: %d %%' % (100 * correct / total))

<a name="Appendix"></a>
# Appendix: Computation Graphs

A *computation graph* is an abstract graphical representation of a mathematical expression involving variables. Each variable is represented as an edge, and each operation as a node; the graph is directed and acyclic. For instance, the expression $(x + y)^2 + x$ can be represented as:

![Computation Graph](https://github.com/harvard-ml-courses/cs181-s21-homeworks/blob/main/hw3/T3_P3_computation_graph.jpg?raw=true)

Expressions can be converted into graphs and vice versa. Computation graphs are useful representations in the context of neural networks, because
1. They encode all the necessary information for the computation of gradients into discrete units through repeated application of the chain rule, and
2. Graphs are easy to implement with code (conceptually, one can imagine storing pointers between nodes)

The process in step #1 is known as *automatic differentiation*.

If you would like to walk through a computation graph on a small neural network, we recommend [this resource](https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/).

# Appendix: Convolutional Neural Networks

A convolutional neural network (CNN) is a special type of neural network that has "convolution layers". CNNs are able to model spatial and temporal dependencies in ordered sequences of input data.

The convolution layer of a CNN applies a filter (sometimes referred to as a "kernel") to an input to produce an output. 

Assume without loss of generality that we have two-dimensional input data $x : m \times n$.  For example, $x$ may be the color of each pixel of a grayscale $m$ by $n$ image.

During the forward pass, we slide (or "convolve") the filter across the input data.  In most CNNs, the filter is a function of "regions" of the data, where individual components of the input are grouped together.  For example, a convolutional filter applied to image data may take into account $3 x 3$ groups of pixels to model dependencies in between adjacent pixels:

![CNN sliding window](https://courses.edx.org/assets/courseware/v1/7396a332c596cb606eae215d96301334/asset-v1:MITx+6.871Jx+2T2021+type@asset+block/cnn_sliding_window.gif)

This GIF image shows a sliding convolutional 3 x 3 filter, being applied to 3 x 3 segments of a 4 x 4 dataset.

source: https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1

Another example of a 1-dimensional convolution filter is illustrated by the below GIF:

![Convolution operation for a simple (1 x 3) filter](https://studio.edx.org/assets/courseware/v1/20da14e015eac1863451dafe3d456e59/asset-v1:MITx+6.871Jx+2T2021+type@asset+block/cnn.gif)

This animation is described below.

In this example, the filter $[2, 3, 1]$ is applied as an inner product with the input vector $x = [x_1, x_2, ..., x_{10}]: 1 \times 10$ which contains annotations of a patient EKG.  Notice here that the filter is applied to 3 successive time-steps of the input vector: the output $y_t$ at time $t$ is equal to $[2, 3, 1] \cdot [x_t, x_{t + 1}, t_{t + 2}]$.  Therefore the output $y_t$ contains information not only about a single EKG annotation $x_t$, but also about nearby EKG annotations $x_{t + 1}$ and $x_{t + 2}$.

We strongly recommend [reading this article](https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/) which walks through how to create a CNN using conv and pooling layers in PyTorch.

If you are interested, you can also watch this [MIT lecture](https://www.youtube.com/watch?v=iaSUYvmCekI) on CNNs.